In [ ]:
%matplotlib inline
import pylab
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
N = 10
x0 = 0.
xf = 1.
g0 = 0.
gf = 0.
h = (xf-x0)/float(N+1)
b = np.zeros(N)
u = np.zeros(N)
A = np.zeros((N,N))

### 2D MPP Jacobi

In [ ]:
def fun(x,y):
    return 100*(x+y)
B = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        B[i,j] = fun(i*h,j*h)
plt.pcolor(B)

In [ ]:
y = np.zeros((N+2,N+2))
for i in range(1,N+1):
    for j in range(1,N+1):
        y[i,j] = fun(i*h,j*h)
    
yold = np.zeros((N+2,N+2))
ynew = np.zeros((N+2,N+2))
plt.pcolor(y[:,:])

In [ ]:
for iterations in range(1000):
    yold = y[:,:]
    for i in range(1,N+1):
        for j in range(1,N+1):
            ay = yold[i-1,j] + yold[i+1,j] + yold[i,j-1] + yold[i,j+1]
            ynew[i,j] = 1/4.*(h**2*fun(i*h,j*h) + ay)
    y = ynew[:,:]
plt.pcolor(y)

## Alternative Direction Implicit

In [ ]:
N=10
h = 1/float(N+1)
x0 = 0.
y0 = 0.
def fun(x,y):
    return 100*(x+y)
A1 = np.zeros((N**2,N**2))
A2 = np.zeros((N**2,N**2))
b = np.zeros(N**2)
u = np.zeros(N**2)

In [ ]:
for i in range(N**2):                        #  Build A1
    for j in range(N**2):
        if i==j:
            A1[i,j] = 2.
        if (i==(j+1)) or ((i+1)==j):
            A1[i,j] = -1.
for i in range(N**2):                        #  Build A2
    for j in range(N**2):
        if i==j:
            A2[i,j] = 2.
        if (i==(j+N)) or ((i+N)==j):
            A2[i,j] = -1.
for j in range(1,N-1):
    for i in range(1,N-1):                    # Building B
        index = i+j*N
        b[index] = h**2*fun(x0 + h*(i+1),y0 + h*(j+1))  

### Test

In [ ]:
b

In [ ]:
A1

In [ ]:
A2

### Build the Bands

In [ ]:
a1 = np.concatenate((np.array([0.]),np.array([A1[i+1,i] for i in range(A1.shape[0]-1)])))
d1 = np.array([A1[i,i] for i in range(A1.shape[0])])
c1 = np.concatenate((np.array([A1[i,i+1] for i in range(A1.shape[0]-1)]),np.array([0.])))

a2 = np.concatenate((np.array([0. for i in range(N)]),np.array([A2[i+N,i] for i in range(A2.shape[0]-N)])))
d2 = np.array([A2[i,i] for i in range(A2.shape[0])])
c2 = np.concatenate((np.array([A2[i,i+N] for i in range(A2.shape[0]-N)]),np.array([0. for i in range(N)])))

In [ ]:
a2

### Tridiagonal Gaussian Elimination and Back Substition

In [ ]:
def GEBS(x,aorig,dorig,corig,borig,N):
    a = np.copy(aorig)
    d = np.copy(dorig)
    c = np.copy(corig)
    b = np.copy(borig)
    u = np.zeros(N**2)
    if x == 1:                           #  Band a distance 1 from diagonal
        for i in range(1,N**2):
            m = a[i]/d[i-1]
            d[i] = d[i] - m*c[i-1]
            b[i] = b[i] - m*b[i-1]
        u[-1] = b[-1]/d[-1]
        for i in range(N**2-1)[::-1]:
            u[i] = (b[i] - c[i]*u[i+1])/d[i]
    if x == 2:                          #  Band a distance N from diagonal
        for i in range(N,N**2):
            m = a[i]/d[i-N]
            d[i] = d[i] - m*c[i-N]
            b[i] = b[i] - m*b[i-N]
        u[-1] = b[-1]/d[-1]
        for i in range(N):                          #  Because of how band back sub works
            u = np.append(u,np.array([0]))
        for i in range(N**2-1)[::-1]:
            u[i] = (b[i] - c[i]*u[i+N])/d[i]
        u = [u[i] for i in range(N**2)]                #  Get rid of extra zeros
    return u

### Test 1

In [ ]:
b = np.array([float(np.random.randint(1,N**2+1)) for i in range(N**2)])
ans = np.copy(b)
y = GEBS(1,a1,d1,c1,b,N)
np.dot(A1,y)-ans

### Test 2

In [ ]:
b = np.array([float(np.random.randint(1,N**2+1)) for i in range(N**2)])
ans = np.copy(b)
y = GEBS(2,a2,d2,c2,b,N)
np.dot(A2,y)-ans

### Peaceman-Rachford

$$(\rho \mathbb{1} + A_1) x^{n+1/2} = b + (\rho \mathbb{1} - A_2)x^n \; \; \; \; \; \; \; \; \; $$        
$$(\rho \mathbb{1} + A_2) x^{n+1} = b + (\rho \mathbb{1} - A_1)x^{n+1/2}\; \; \; \;  \; \; $$       

#### Initialize

In [ ]:
N=10
rho = 1.75                           # Pick a parameter
x = np.zeros(N**2)
b = np.zeros(N**2)
for j in range(1,N-1):
    for i in range(1,N-1):                    # Building B
        index = i+j*N
        b[index] = h**2*fun(x0 + h*(i+1),y0 + h*(j+1)) 
x = b[:]
#  New Bands
a1 = np.concatenate((np.array([0.]),np.array([-1. for i in range(N**2-1)])))
d1 = np.array([2. for i in range(N**2)])
c1 = np.concatenate((np.array([-1. for i in range(N**2-1)]),np.array([0.])))

a2 = np.concatenate((np.array([0. for i in range(N)]),np.array([-1. for i in range(N**2-N)])))
d2 = np.array([2. for i in range(N**2)])
c2 = np.concatenate((np.array([-1. for i in range(N**2-N)]),np.array([0. for i in range(N)])))

for i in range(N**2):
    d1[i] = d1[i] + rho            # Specifically for PR
    d2[i] = d2[i] + rho

#### PR Iteration

In [ ]:
for i in range(10):
    rhs = b + np.dot((rho*np.identity(N**2) - A2),x)
    x = GEBS(1,a1,d1,c1,rhs,N)
    rhs = b + np.dot((rho*np.identity(N**2) - A1),x)
    x = GEBS(2,a2,d2,c2,rhs,N)

### Plot

In [ ]:
y = np.zeros((N+1,N+1))
for i in range(1,N):
    for j in range(1,N):
        y[i,j] = x[j + N*i]

In [ ]:
plt.pcolor(y[:,:])

### Banded Multiplication


In [ ]:
N=5
A = np.zeros((N**2,N**2))
A1 = np.zeros((N**2,N**2))
A2 = np.zeros((N**2,N**2))
b = np.array([np.random.randint(10) for i in range(N**2)])

for i in range(N**2):                        #  Build A1
    for j in range(N**2):
        if i==j:
            A1[i,j] = 2.
        if (i==(j+1)) or ((i+1)==j):
            A1[i,j] = -1.
for i in range(N**2):                        #  Build A2
    for j in range(N**2):
        if i==j:
            A2[i,j] = 2.
        if (i==(j+N)) or ((i+N)==j):
            A2[i,j] = -1.
            
a1 = np.concatenate((np.array([0.]),np.array([-1. for i in range(N**2-1)])))
d1 = np.array([2. for i in range(N**2)])
c1 = np.concatenate((np.array([-1. for i in range(N**2-1)]),np.array([0.])))

a2 = np.concatenate((np.array([0. for i in range(N)]),np.array([-1. for i in range(N**2-N)])))
d2 = np.array([2. for i in range(N**2)])
c2 = np.concatenate((np.array([-1. for i in range(N**2-N)]),np.array([0. for i in range(N)])))



In [ ]:
b

In [ ]:
def bandedmult(x,a,d,c,b,N):
    if x == 1:
        vec = np.append(np.append(0,b),0)
        u = np.zeros(N**2 + 2)
        for i in range(1,N**2+1):
            u[i] = a[i-1]*vec[i-1] + d[i-1]*vec[i] + c[i-1]*vec[i+1]
        u = [u[i] for i in range(1,N**2+1)]
    if x == 2:
        vec = np.append(np.append([0. for i in range(N)],b),[0. for i in range(N)])
        u = np.zeros(N**2 + 2*N)
        for i in range(N,N**2+N):
            u[i] = a[i-N]*vec[i-N] + d[i-N]*vec[i] + c[i-N]*vec[i+N]
        u = [u[i] for i in range(N,N**2+N)]
    return u

In [ ]:
bandedmult(1,a1,d1,c1,b,N) - np.dot(A1,b)

In [ ]:
bandedmult(2,a2,d2,c2,b,N) - np.dot(A2,b)